# API Key & Model Selection

The following cell lets you test that you're API key is accessible. To add your API key, follow instructions in `README`.

> 💡 Remember! This cell needs to be ran before you can run any other cells that utilize the `API_KEY` variable. If you reset the runtime, you'll need to run this cell again.

We'll also create a constant to choose our model here so don't have to repeat it throughout every cell.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
API_KEY = os.getenv('OPENAI_API_KEY')

print(API_KEY)

MODEL = 'gpt-4o-mini'

# Getting Started

This code snippet is from the [Text generation and prompting](https://platform.openai.com/docs/guides/text?api-mode=responses) API reference of the OpenAI documentation. Try it out to make sure everything is working so far.

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=API_KEY)

response = client.responses.create(
    model=MODEL,
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

# Part 1: Create a Chat Loop

Use the [Text generation and prompting](https://platform.openai.com/docs/guides/text?api-mode=responses) guide to complete the following todos.

> 💡 If you navigate to the documentation yourself, make sure that you're using the "Responses" API, not "Chat Completions". Both work, but "Responses" is newer, and it's what we'll use in examples and solutions.

<details>
<summary> 👉 Hints! </summary>

- You'll need to study the examples to see how to include multiple messages, or "conversation state".
- The `input` function works in Jupyter Notebooks just like it does in a `.py` file to get input from the user.
- Your `chat` function will need to run in a loop. Make sure to provide a way for the user to exit the loop.

</details>

In [ ]:
client = OpenAI(api_key=API_KEY)

def get_response(history):
    response = client.responses.create(
        model=MODEL,
        input=history,
    )
    return response.output_text


def chat():
    """Main chat loop."""
    print("Welcome to ChatGPT-like Application!")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("-" * 50)

    history = []

    # Add a system message to set the assistant's behavior
    system_msg = { 'role': 'developer', 'content': 'You are a helpful assistant.' }
    history.append(system_msg)

    while True:
        # Get user input
        user_input = input('You: ')

        # Check if user wants to exit
        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break
    
        print(f'You: {user_input}')

        # Add user message to history
        user_msg = { 'role': 'user', 'content': user_input }
        history.append(user_msg)

        # Display thinking indicator
        print("Thinking...", end="\r")

        # Generate and display response
        response = get_response(history)

        print("Assistant:", response)

        # Add assistant message to history
        assistant_msg = { 'role': 'assistant', 'content': response }
        history.append(assistant_msg)

        print("-" * 50)

chat()

# Part 2: Add Classes to Your Program

Managing the messages and history of our conversation is a bit clumsy. Create some classes with whatever methods necessary to make our code more declarative. Use these suggestions or take your own approach:

- `Message`: Have a role and content. Takes the form of a `dict` object for API calls.
- `ChatHistory`: Maintains an array of `Message` objects. Takes the form of a `list` for API calls.
- `ChatManager`: Orchestrates the interaction utilizing `Message` and `ChatHistory` classes.

In [ ]:
class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

    def to_dict(self):
        return {'role': self.role, 'content': self.content}


class ChatHistory:
    def __init__(self):
        self.messages = []

    def add(self, message):
        self.messages.append(message)

    def to_list(self):
        return [msg.to_dict() for msg in self.messages]


class ChatManager:
    def __init__(self, model, system_prompt):
        self.model = model
        self.history = ChatHistory()
        self.history.add(Message('developer', system_prompt))
        self.client = OpenAI(api_key=API_KEY)

    def get_response(self):
        response = self.client.responses.create(
            model=self.model,
            input=self.history.to_list()
        )
        return response.output_text

    def run(self):
        print("Welcome to ChatGPT-like Application!")
        print("Type 'exit' or 'quit' to end the conversation.")
        print("-" * 50)

        while True:
            user_input = input('You: ')
            if user_input.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
        
            print(f'You: {user_input}')

            self.history.add(Message('user', user_input))
            print("Thinking...", end="\r")

            response_text = self.get_response()
            print("Assistant:", response_text)

            self.history.add(Message('assistant', response_text))
            print("-" * 50)


SYSTEM_PROMPT = "You are a helpful assistant."
chat = ChatManager(MODEL, SYSTEM_PROMPT)
chat.run()

# Part 3: Token Management

When making API calls, we're charged per-token. Let's make sure that our user can't surpass a certain number of tokens in their message.

First, skim this information on [Managing Tokens](https://platform.openai.com/docs/advanced-usage#managing-tokens). Towards the end of this section, they mention the Python library [tiktoken](https://github.com/openai/tiktoken).

Using the documentation for tiktoken (the README file of the tiktoken GitHub repo), and the [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb), implement a feature that stops conversations from exceeding a certain token limit. Make sure to test that the result you're getting from `tiktoken` is close to the actual tokens being used, which you can find in the API response.

> 💡 You'll need to use `poetry add tiktoken` to update your project with the new library.

In [ ]:
import tiktoken

class TokenManager:
    def __init__(self, model, max_tokens):
        self.encoding = tiktoken.encoding_for_model(model)
        self.max_tokens = max_tokens
        self.tokens_used = 0

    def exceeds_limit(self):
        return self.tokens_used >= self.max_tokens

    def add_tokens(self, messages):
        num_tokens = 0
        tokens_per_message = 3
        tokens_per_name = 1

        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(self.encoding.encode(value))
                if key == 'name':
                    num_tokens += tokens_per_name

        num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

        self.tokens_used += num_tokens

    def reset(self):
        self.tokens_used = 0


# No changes to `Message` class
class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

    def to_dict(self):
        return {'role': self.role, 'content': self.content}


# Only change to `ChatHistory` was adding a `clear` method
class ChatHistory:
    def __init__(self):
        self.messages = []

    def add(self, message):
        self.messages.append(message)

    def to_list(self):
        return [msg.to_dict() for msg in self.messages]

    def clear(self):
      self.messages = []


class ChatManager:
    def __init__(self, model, system_prompt):
        self.model = model
        self.history = ChatHistory()
        self.system_prompt = system_prompt
        self.history.add(Message('developer', system_prompt))
        self.client = OpenAI(api_key=API_KEY)
        self.token_manager = TokenManager(model, 300) # Low token limit for testing

    def get_response(self):
        response = self.client.responses.create(
            model=self.model,
            input=self.history.to_list()
        )
        return response.output_text

    def run(self):
        print("Welcome to ChatGPT-like Application!")
        print("Type 'exit' or 'quit' to end the conversation.")
        print("-" * 50)

        while True:
            user_input = input('You: ')
            if user_input.lower() in ['exit', 'quit']:
                print("Goodbye!")
                break
        
            print(f'You: {user_input}')

            self.history.add(Message('user', user_input))
            self.token_manager.add_tokens(self.history.to_list())

            if self.token_manager.exceeds_limit():
                print("Conversation limit reached. Please start a new conversation.")
                print("Clearing conversation history...")
                print("-" * 50)
                print("Welcome to ChatGPT-like Application!")
                print("Type 'exit' or 'quit' to end the conversation.")
                print("-" * 50)

                self.history.clear()
                self.token_manager.reset()
                self.history.add(Message('developer', self.system_prompt))
                continue

            print("Thinking...", end="\r")

            response_text = self.get_response()
            print("Assistant:", response_text)

            self.history.add(Message('assistant', response_text))
            print("-" * 50)


SYSTEM_PROMPT = "You are a helpful assistant."
chat = ChatManager(MODEL, SYSTEM_PROMPT)
chat.run()

# Bonus Ideas

If you have extra time, here are some ideas:

- **Customize the system prompt**. Allow the user to either select some attributes, like 'formal' vs 'informal', or let them set their own system prompt.

- **Save past conversations to a file.** Create a directory that stores conversations once they've ended, along with meta-data like the date and time of the conversation.

- **Implement a 'help' command.** Give the user a way to access a help menu with more information. You could allow for other commands, like checking token limits.

- **Timeout the Chatbot**. After a certain amount of inactivity, automatically end the conversation.

- **Error Handling**. We don't have any error handling right now. Look into possible responses from the OpenAI API and handle them gracefully. You can view information about [the response object here](https://platform.openai.com/docs/api-reference/responses/object).